# ResNet-34 Architecture

ResNet-34, short for Residual Network with 34 layers, is a deep convolutional neural network (CNN) designed to address the vanishing gradient problem that hampers the training of deep networks. Introduced by Kaiming He et al. in their paper ["Deep Residual Learning for Image Recognition"](https://arxiv.org/abs/1512.03385), ResNet-34 utilizes "skip connections" or "residual connections" which allow for easier gradient flow through the network.

## Key Features

### Residual Blocks

Instead of learning the underlying mapping, residual blocks learn the residuals (or the difference) between the input and output. This is done by introducing shortcut connections that skip one or more layers.

A residual block typically has the following structure:
- Two 3x3 convolutional layers.
- Batch normalization (BN) after each convolutional layer.
- ReLU activation function after each BN.
- A shortcut connection that adds the input of the block to the output.

### Skip Connections

These connections bypass one or more layers by performing identity mapping and then adding the output to the skipped layer’s output. This helps mitigate the vanishing gradient problem by allowing gradients to flow directly through the network.

### Network Depth and Layer Composition

ResNet-34 is composed of 34 layers, which include:
- **Convolutional Layers**: Perform feature extraction by applying filters to the input.
- **Pooling Layers**: Reduce the spatial dimensions (height and width) of the input, keeping the depth the same.
- **Fully Connected Layers**: Perform classification tasks based on the extracted features.

### Layer Breakdown

1. **Initial Convolution and Pooling**:
   - 1 Convolution layer with 64 filters of size 7x7, stride 2.
   - 1 MaxPooling layer with size 3x3, stride 2.

2. **Residual Blocks**:
   - **Conv2_x**: 3 blocks, each with 2 convolutional layers of size 3x3 and 64 filters.
   - **Conv3_x**: 4 blocks, each with 2 convolutional layers of size 3x3 and 128 filters. The first block uses a convolutional layer with stride 2 to reduce spatial dimensions.
   - **Conv4_x**: 6 blocks, each with 2 convolutional layers of size 3x3 and 256 filters. The first block uses a convolutional layer with stride 2 to reduce spatial dimensions.
   - **Conv5_x**: 3 blocks, each with 2 convolutional layers of size 3x3 and 512 filters. The first block uses a convolutional layer with stride 2 to reduce spatial dimensions.

3. **Final Layers**:
   - 1 Average Pooling layer with size 7x7.
   - 1 Fully Connected layer with 1000 neurons (for classification into 1000 classes).

### Detailed Architecture



In [5]:
# CREATE A RESIDUAL UNIT LAYER
from functools import partial
import tensorflow as tf

DefaultConv2d = partial(tf.keras.layers.Conv2D, kernel_size=3, strides=1, padding="same",
                        kernel_initializer="he_normal", use_bias=False)

In [6]:
class ResidualUnit(tf.keras.layers.Layer):
  def __init__(self, filters, strides=1, activation="relu", **kwargs):
    super().__init__(**kwargs)
    self.activation = tf.keras.activations.get(activation)
    self.main_layers = [
        DefaultConv2d(filters, strides=strides),
        tf.keras.layers.BatchNormalization(),
        self.activation,
        DefaultConv2d(filters),
        tf.keras.layers.BatchNormalization()
    ]

    self.skip_layers = []
    if strides > 1:
      self.skip_layers = [
          DefaultConv2d(filters, kernel_size=1, strides=strides),
          tf.keras.layers.BatchNormalization()
      ]

      def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
          Z = layer(Z)

        skip_Z = inputs
        for layer in self.skip_layers:
          skip_Z = layer(skip_Z)

        return self.activation(Z + skip_Z)

In [9]:
# BUILDING RESNET-34 USING SEQUENTIAL

model = tf.keras.Sequential([
    DefaultConv2d(64, kernel_size=7, strides=2, input_shape=[224, 224, 3]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same")
])

prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
  strides = 1 if filters == prev_filters else 2
  model.add(ResidualUnit(filters, strides=strides))
  prev_filters = filters

model.add(tf.keras.layers.GlobalAvgPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation="softmax"))